In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [ ]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
ActiGraph_60s_epoch_proc_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/processed_data/actigraph/epoch_data/60s_ref'
ActiGraph_60hz_raw_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/ActiGraph_raw'

# sleep time input_data
sleep_time_file = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/metadata/Copy of record_availability_final.csv'

# Output
Actigraph_output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed_graph'

### Load Sleep Data

In [ ]:
raw_files = [f for f in listdir(ActiGraph_60hz_raw_folder) if isfile(join(ActiGraph_60hz_raw_folder, f))]
epoch_files = [f for f in listdir(ActiGraph_60s_epoch_proc_folder) if isfile(join(ActiGraph_60s_epoch_proc_folder, f))]

data_dict = {f.split()[0]: {'raw': join(ActiGraph_60hz_raw_folder, f)} for f in raw_files}
for ef in epoch_files:
  key = ef.split()[0]
  if key in data_dict.keys():
    data_dict[key]['epoch'] = join(ActiGraph_60s_epoch_proc_folder, ef)

In [ ]:
print(data_dict['OA46'])

{'raw': '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/ActiGraph_raw/OA46 fixedRAW.csv', 'epoch': '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/processed_data/actigraph/epoch_data/60s_ref/OA46 fixed60sec_60sREF.csv'}


In [ ]:
sleep_df = pd.read_csv(sleep_time_file)
sleep_df.head(2)

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,begin_time,end_time
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-06 15:00:00,2019-05-06 22:00:00
1,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-07 06:00:00,2019-05-07 22:00:00


### Generate statistical features

In [ ]:
FREQUENCY = 60 #HZ amount of the accelerometer
REFERENCE_EPOCH = 60 #Epoch value that EE is given
TIME_EPOCH_DICT = {
    # 'Epoch1': FREQUENCY * 1,
    # 'Epoch5': FREQUENCY * 5,
    'Epoch30': FREQUENCY * 30,
    # 'Epoch60': FREQUENCY * 60
}

In [ ]:
df1 = pd.DataFrame()
x = np.linspace(0,1000,100)
df1['x'] = x

y = np.linspace(0,500,100)
df1['y'] = y

z = np.linspace(0,3000,100)
df1['z'] = z

vm = np.sqrt([(df1['x'] ** 2) + (df1['y'] ** 2) + (df1['z'] ** 2)])[0]
df1['vm']= vm
# x_y = df1['x'] + df1['y']
# df1['x_y'] = x_y
# df1['y_square'] = df1['y']*df1['y']
# df1['z_square'] = df1['z']*df1['z']
# df1['yz_square'] = (df1['z']*df1['z'])+(df1['y']*df1['y'])
# df1['yz_sqrt'] = np.sqrt((df1['z']*df1['z'])+(df1['y']*df1['y']))
orientation_roll = np.arctan2(df1['y'],df1['z'])*180/math.pi
# root_yz = math.sqrt((df1['y']**2)+(df1['z']**2))
orientation_pitch = np.arctan2(-df1['x'],np.sqrt((df1['z']*df1['z'])+(df1['y']*df1['y'])))*180/math.pi
angle = (90 * np.arcsin(df1['x'] / df1['vm'])) / (math.pi / 2)
df1['orientation_roll'] = orientation_roll
df1['orientation_pitch'] = orientation_pitch
df1['angle'] = angle

gdf = df1.groupby(df1.index//10)

for name,group in gdf:
  print(name)
  print(group)



# gm = gdf['vals'].mean()
# print(gm)

# print("***")

# aggregated_gm = pd.DataFrame()
# aggregated_gm['mean_val'] = gdf['vals'].mean()
# aggregated_gm['max'] = gdf['vals'].max()
# aggregated_gm['min'] = gdf['vals'].min()



# print(aggregated_gm)




0
           x          y  ...  orientation_pitch     angle
0   0.000000   0.000000  ...           -0.00000       NaN
1  10.101010   5.050505  ...          -18.20076  18.20076
2  20.202020  10.101010  ...          -18.20076  18.20076
3  30.303030  15.151515  ...          -18.20076  18.20076
4  40.404040  20.202020  ...          -18.20076  18.20076
5  50.505051  25.252525  ...          -18.20076  18.20076
6  60.606061  30.303030  ...          -18.20076  18.20076
7  70.707071  35.353535  ...          -18.20076  18.20076
8  80.808081  40.404040  ...          -18.20076  18.20076
9  90.909091  45.454545  ...          -18.20076  18.20076

[10 rows x 7 columns]
1
             x          y  ...  orientation_pitch     angle
10  101.010101  50.505051  ...          -18.20076  18.20076
11  111.111111  55.555556  ...          -18.20076  18.20076
12  121.212121  60.606061  ...          -18.20076  18.20076
13  131.313131  65.656566  ...          -18.20076  18.20076
14  141.414141  70.707071  ...     

In [ ]:
for _, row in tqdm(sleep_df.iterrows(), total=sleep_df.shape[0]):

  user_id = row['Participant']
  begin_time = row['begin_time']
  end_time = row['end_time']

  age = row['age_yr']
  sex = row['sex']
  BMI = row['BMI']
  mass = row['bodymass_kg']

  height = (mass/BMI)**0.5

  if('HOA1' != user_id):
    continue
  print(user_id)

  # get respective epoch dataset
  epoch_df = pd.read_csv(data_dict[user_id]['epoch'])
  epoch_df = epoch_df[(begin_time < epoch_df['Time_stamp']) & (epoch_df['Time_stamp'] < end_time)]
  # print(list(epoch_df.index)[0])
  # continue
  # get respective raw dataset
  raw_begin_index = list(epoch_df.index)[0] * REFERENCE_EPOCH * FREQUENCY
  raw_data_length = epoch_df.shape[0] * REFERENCE_EPOCH * FREQUENCY
  raw_df_orig = pd.read_csv(data_dict[user_id]['raw'], skiprows=11+raw_begin_index, nrows=raw_data_length, header=None)
  raw_df_orig.columns = ['Timestamp', 'X', 'Y', 'Z']
  # print(raw_df_orig.head(20))

  """
  Process RAW data as required by the model features
  """

  # VM and MANGLE needs for calculations by Staudenmayer model
  # Calculate the vector magnitude from X, Y, Z raw readings
  # Calculate the angle of arcsin from X and VM, arcsin(axis used/vector magnitude)/(pi/2)
  raw_df_orig['vm'] = np.sqrt([(raw_df_orig.X ** 2) + (raw_df_orig.Y ** 2) + (raw_df_orig.Z ** 2)])[0]
  # raw_df_orig['angle'] = (90 * np.arcsin(raw_df_orig.X / raw_df_orig['vm'])) / (math.pi / 2)
  # print(raw_df_orig['vm'].head(20))
  # Calculate ENMO (Euclidean Norm Minus One) reguired for Hilderbrand model
  raw_df_orig['enmo'] = np.sqrt([(raw_df_orig.X ** 2) + (raw_df_orig.Y ** 2) + (raw_df_orig.Z ** 2)])[0] - 1
  raw_df_orig.loc[raw_df_orig.enmo < 0, 'enmo'] = 0
  # print(raw_df_orig['enmo'].head(20))

  
  # For each epoch given
  for epoch_name, epoch_duration in TIME_EPOCH_DICT.items():
    print(epoch_name,epoch_duration)
    
    # continue
    # Create output path for the combined file
    output_file_name = join(Actigraph_output_folder, epoch_name,'{}_{}_to_{}_{}.csv'.format(user_id, begin_time, end_time, epoch_name).replace(':', '-').replace(' ', '_'))
    if not exists(join(Actigraph_output_folder, epoch_name)):
      makedirs(join(Actigraph_output_folder, epoch_name))
    
    # Only run for non processed files
    # if exists(output_file_name):
    #   continue

    # Copy raw data
    raw_df = raw_df_orig.copy()
    # print(raw_df.head(10))
    # print(raw_df.index)
    raw_df.reset_index(drop=True, inplace=True)
    # print(raw_df.head(10))
    # print(raw_df.index)
    

    # X shifted needs to calculate covariance for Montoye 2017 model
    raw_df['X_shifted'] = raw_df_orig['X'].shift(epoch_duration)
    raw_df['Y_shifted'] = raw_df_orig['Y'].shift(epoch_duration)
    raw_df['Z_shifted'] = raw_df_orig['Z'].shift(epoch_duration)

    """
    Calculate the statistical inputs (Features)
    """
    # print(raw_df.index // epoch_duration)
    
    hip_grouped_temp = raw_df.groupby(raw_df.index // epoch_duration)
    # k=0
    # for name,group in hip_grouped_temp:
    #   print(name)
    #   print(group)
    #   k=k+1
    #   if(k==100):break
    # break
    aggregated_hip = pd.DataFrame()

    # General stats
    aggregated_hip['mvm'] = hip_grouped_temp['vm'].mean()

    # Aggregated values required for Hilderbrand model
    aggregated_hip['enmo'] = hip_grouped_temp['enmo'].mean()
    aggregated_hip['enmo'] = aggregated_hip['enmo'].fillna(1)

    # Aggregated values required for Montoye Models
    aggregated_hip['XMean'] = hip_grouped_temp['X'].mean()
    aggregated_hip['YMean'] = hip_grouped_temp['Y'].mean()
    aggregated_hip['ZMean'] = hip_grouped_temp['Z'].mean()
    aggregated_hip['XVar'] = hip_grouped_temp['X'].var()
    aggregated_hip['YVar'] = hip_grouped_temp['Y'].var()
    aggregated_hip['ZVar'] = hip_grouped_temp['Z'].var()

    aggregated_hip['X10perc'] = hip_grouped_temp['X'].quantile(.1)
    aggregated_hip['X25perc'] = hip_grouped_temp['X'].quantile(.25)
    aggregated_hip['X50perc'] = hip_grouped_temp['X'].quantile(.5)
    aggregated_hip['X75perc'] = hip_grouped_temp['X'].quantile(.75)
    aggregated_hip['X90perc'] = hip_grouped_temp['X'].quantile(.9)

    aggregated_hip['Y10perc'] = hip_grouped_temp['Y'].quantile(.1)
    aggregated_hip['Y25perc'] = hip_grouped_temp['Y'].quantile(.25)
    aggregated_hip['Y50perc'] = hip_grouped_temp['Y'].quantile(.5)
    aggregated_hip['Y75perc'] = hip_grouped_temp['Y'].quantile(.75)
    aggregated_hip['Y90perc'] = hip_grouped_temp['Y'].quantile(.9)

    aggregated_hip['Z10perc'] = hip_grouped_temp['Z'].quantile(.1)
    aggregated_hip['Z25perc'] = hip_grouped_temp['Z'].quantile(.25)
    aggregated_hip['Z50perc'] = hip_grouped_temp['Z'].quantile(.5)
    aggregated_hip['Z75perc'] = hip_grouped_temp['Z'].quantile(.75)
    aggregated_hip['Z90perc'] = hip_grouped_temp['Z'].quantile(.9)

    aggregated_hip['X_cov'] = hip_grouped_temp.apply(lambda x: x['X'].cov(x['X_shifted']))
    aggregated_hip['Y_cov'] = hip_grouped_temp.apply(lambda x: x['Y'].cov(x['Y_shifted']))
    aggregated_hip['Z_cov'] = hip_grouped_temp.apply(lambda x: x['Z'].cov(x['Z_shifted']))

    aggregated_hip['X_cov'] = aggregated_hip['X_cov'].fillna(0)
    aggregated_hip['Y_cov'] = aggregated_hip['Y_cov'].fillna(0)
    aggregated_hip['Z_cov'] = aggregated_hip['Z_cov'].fillna(0)

    aggregated_hip['X_min'] = hip_grouped_temp['X'].min()
    aggregated_hip['Y_min'] = hip_grouped_temp['Y'].min()
    aggregated_hip['Z_min'] = hip_grouped_temp['Z'].min()
    aggregated_hip['X_max'] = hip_grouped_temp['X'].max()
    aggregated_hip['Y_max'] = hip_grouped_temp['Y'].max()
    aggregated_hip['Z_max'] = hip_grouped_temp['Z'].max()

    """
    Repeat rows of epoch file as required to supplement raw data.
    """
    # Number of rows for each epoch
    epoch_df_current = epoch_df.copy()
    # print(epoch_df_current.head())
    epoch_df_current.reset_index(drop=True, inplace=True)
    # print(epoch_df_current.head())
    
    multiplication_factor = int(REFERENCE_EPOCH / (epoch_duration/FREQUENCY))#ex: 60/(1800/60) for 30 epochs, ultimately 60/30
    # print('mul',[multiplication_factor,REFERENCE_EPOCH,epoch_duration,FREQUENCY])
    print('before',epoch_df_current.head())
    epoch_df_current = epoch_df_current.reindex(np.repeat(epoch_df_current.index.values, multiplication_factor), method='ffill')
    print('after',epoch_df_current.head())
    epoch_df_current['waist_ee_final'] = epoch_df_current['waist_ee']/multiplication_factor
    print('EE',epoch_df_current['waist_ee'])

    epoch_df_current.reset_index(drop=True, inplace=True)
    epoch_df_current['sex'] = sex
    epoch_df_current['height'] = height
    epoch_df_current['age'] = age
    epoch_df_current['BMI'] = BMI

 
    
    # break

    #user_characteristics


    # Combine epoch data with raw data
    # Hilderbrand
    epoch_df_current['mvm'] = aggregated_hip['mvm']
    epoch_df_current['enmo'] = aggregated_hip['enmo']


    # Montoye
    epoch_df_current['raw_XMean'] = aggregated_hip['XMean']
    epoch_df_current['raw_YMean'] = aggregated_hip['YMean']
    epoch_df_current['raw_ZMean'] = aggregated_hip['ZMean']

    epoch_df_current['raw_XVar'] = aggregated_hip['XVar']
    epoch_df_current['raw_YVar'] = aggregated_hip['YVar']
    epoch_df_current['raw_ZVar'] = aggregated_hip['ZVar']

    epoch_df_current['raw_X10perc'] = aggregated_hip['X10perc']
    epoch_df_current['raw_X25perc'] = aggregated_hip['X25perc']
    epoch_df_current['raw_X50perc'] = aggregated_hip['X50perc']
    epoch_df_current['raw_X75perc'] = aggregated_hip['X75perc']
    epoch_df_current['raw_X90perc'] = aggregated_hip['X90perc']

    epoch_df_current['raw_Y10perc'] = aggregated_hip['Y10perc']
    epoch_df_current['raw_Y25perc'] = aggregated_hip['Y25perc']
    epoch_df_current['raw_Y50perc'] = aggregated_hip['Y50perc']
    epoch_df_current['raw_Y75perc'] = aggregated_hip['Y75perc']
    epoch_df_current['raw_Y90perc'] = aggregated_hip['Y90perc']

    epoch_df_current['raw_Z10perc'] = aggregated_hip['Z10perc']
    epoch_df_current['raw_Z25perc'] = aggregated_hip['Z25perc']
    epoch_df_current['raw_Z50perc'] = aggregated_hip['Z50perc']
    epoch_df_current['raw_Z75perc'] = aggregated_hip['Z75perc']
    epoch_df_current['raw_Z90perc'] = aggregated_hip['Z90perc']

    # epoch_df_current['raw_X_cov'] = aggregated_hip['X_cov']
    # epoch_df_current['raw_Y_cov'] = aggregated_hip['Y_cov']
    # epoch_df_current['raw_Z_cov'] = aggregated_hip['Z_cov']

    print('epoch_df', epoch_df.shape)
    print('epoch_df_current', epoch_df_current.shape, multiplication_factor)
    print('raw', aggregated_hip.shape)
    print('ag_hip', aggregated_hip.head())
    print('after',epoch_df_current.head())

    epoch_df_current.to_csv(output_file_name, index=None)












  0%|          | 0/1351 [00:00<?, ?it/s]

HOA1
Epoch30 1800












  0%|          | 1/1351 [00:05<2:13:41,  5.94s/it]

before    AxisY  AxisX  AxisZ  ... waist_vm_cpm  waist_cpm  waist_ee
0    938    169   2570  ...  2741.040861        938  3.034394
1    911    167   2299  ...  2478.550181        911  2.807865
2    597    144   2176  ...  2261.000000        597  1.988053
3     91    100    377  ...   400.512172         91  1.435501
4    738    213   1679  ...  1846.362370        738  2.142025

[5 rows x 8 columns]
after    AxisY  AxisX  AxisZ  ... waist_vm_cpm  waist_cpm  waist_ee
0    938    169   2570  ...  2741.040861        938  3.034394
0    938    169   2570  ...  2741.040861        938  3.034394
1    911    167   2299  ...  2478.550181        911  2.807865
1    911    167   2299  ...  2478.550181        911  2.807865
2    597    144   2176  ...  2261.000000        597  1.988053

[5 rows x 8 columns]
EE 0      3.034394
0      3.034394
1      2.807865
1      2.807865
2      1.988053
         ...   
416    1.722697
417    3.191018
417    3.191018
418    1.515217
418    1.515217
Name: waist_ee, Leng











  0%|          | 2/1351 [00:20<3:08:30,  8.38s/it]

...











  1%|          | 7/1351 [01:39<5:35:54, 15.00s/it]

HOA1
Epoch30 1800












  1%|          | 8/1351 [01:47<4:47:55, 12.86s/it]

before    AxisY  AxisX  AxisZ  ... waist_vm_cpm  waist_cpm  waist_ee
0    655    450   2623  ...  2740.739681        655  3.034134
1     25     66    104  ...   125.686117         25  1.363429
2      0    313    130  ...   338.923295          0  1.336129
3      0     10     91  ...    91.547802          0  1.336129
4      0      0     41  ...    41.000000          0  1.336129

[5 rows x 8 columns]
after    AxisY  AxisX  AxisZ  ... waist_vm_cpm  waist_cpm  waist_ee
0    655    450   2623  ...  2740.739681        655  3.034134
0    655    450   2623  ...  2740.739681        655  3.034134
1     25     66    104  ...   125.686117         25  1.363429
1     25     66    104  ...   125.686117         25  1.363429
2      0    313    130  ...   338.923295          0  1.336129

[5 rows x 8 columns]
EE 0      3.034134
0      3.034134
1      1.363429
1      1.363429
2      1.336129
         ...   
116    1.658269
117    3.555912
117    3.555912
118    4.093533
118    4.093533
Name: waist_ee, Leng











 40%|████      | 542/1351 [01:47<2:01:24,  9.00s/it]









100%|██████████| 1351/1351 [01:47<00:00, 12.57it/s]


In [ ]:
x,y,z = 0.625,0.543,-0.555
vm = ((x**2)+(y**2)+(z**2))**0.5
print('vm',vm)
angle_r = (90 * np.arcsin(x / vm)) / (math.pi / 2)
print('angle_r',angle_r)

angle_roll = np.arctan2(y,z)*(180/math.pi)
angle_pitch = np.arctan2(x,math.sqrt((y*y) +(z*z)))*(180/math.pi)
print('roll',angle_roll)
print('pitch',angle_pitch)

vm 0.9967441998827985
angle_r 38.83218710183141
roll 135.6261584535352
pitch 38.8321871018314


### Helper code

In [ ]:
# Helper code
sorted(raw_files)[0]
pd.read_csv(join(ActiGraph_60hz_raw_folder, sorted(raw_files)[0]), skiprows=11+(420*60*60), nrows=100, header=None)

In [ ]:
epoch_files = [f for f in listdir(ActiGraph_60s_epoch_proc_folder) if isfile(join(ActiGraph_60s_epoch_proc_folder, f))]
print(epoch_files)

['HOA1 (2019-05-06)60sec_60sREF.csv', 'HOA2 (2019-05-06)60sec_60sREF.csv', 'HOA3 (2019-05-06)60sec_60sREF.csv', 'HOA4 (2019-05-06)60sec_60sREF.csv', 'HOA10 (2019-06-24)60sec_60sREF.csv', 'HOA5 (2019-05-20)60sec_60sREF.csv', 'HOA11 (2019-07-01)60sec_60sREF.csv', 'HOA6 (2019-05-20)60sec_60sREF.csv', 'HOA12 (2019-07-01)60sec_60sREF.csv', 'HOA7 (2019-05-27)60sec_60sREF.csv', 'HOA14 (2019-07-15)60sec_60sREF.csv', 'HOA8 (2019-05-27)60sec_60sREF.csv', 'HOA18 (2019-07-29)60sec_60sREF.csv', 'HOA15 (2019-07-29)60sec_60sREF.csv', 'HOA9 (2019-05-27)60sec_60sREF.csv', 'HOA19 (2019-08-05)60sec_60sREF.csv', 'HOA16 (2019-07-29)60sec_60sREF.csv', 'HOA24 (2019-08-26)60sec_60sREF.csv', 'HOA20 (2019-08-19)60sec_60sREF.csv', 'HOA17 (2019-07-29)60sec_60sREF.csv', 'HOA25 (2019-09-02)60sec_60sREF.csv', 'HOA29 (2019-09-02)60sec_60sREF.csv', 'HOA21 (2019-08-19)60sec_60sREF.csv', 'HOA26 (2019-09-02)60sec_60sREF.csv', 'HOA36 (2019-09-30)60sec_60sREF.csv', 'HOA30 (2019-09-02)60sec_60sREF.csv', 'HOA22 (2019-08-19)6

In [ ]:
epoch_files[0]

'HOA1 (2019-05-06)60sec_60sREF.csv'

In [ ]:
hoa1_df = pd.read_csv(join(ActiGraph_60s_epoch_proc_folder, epoch_files[0]))

In [ ]:
hoa1_df.head()

,AxisY,AxisX,AxisZ,Time_stamp,waist_vm_60,waist_vm_cpm,waist_cpm,waist_ee
0,1000,226,2559,2019-05-06 15:00:00,2756.729403,2756.729403,1000,3.047933
1,938,169,2570,2019-05-06 15:01:00,2741.040861,2741.040861,938,3.034394
2,911,167,2299,2019-05-06 15:02:00,2478.550181,2478.550181,911,2.807865
3,597,144,2176,2019-05-06 15:03:00,2261.000000,2261.000000,597,1.988053
4,91,100,377,2019-05-06 15:04:00,400.512172,400.512172,91,1.435501


In [ ]:
a1 = '2019-05-06 22:00:00'
a2 = '2019-05-07 06:00:00'

In [ ]:
hoa1_df[hoa1_df['Time_stamp'] < a1]

In [ ]:
for _, row in sleep_df.iterrows():

  user_id = row['Participant']
  begin_time = row['begin_time']
  end_time = row['end_time']

  print(user_id, begin_time, end_time)

  # get respective epoch dataset
  epoch_df = pd.read_csv(data_dict[user_id]['epoch'])
  epoch_df = epoch_df[(begin_time < epoch_df['Time_stamp']) & (epoch_df['Time_stamp'] < end_time)]

  # get respective raw dataset
  raw_begin_index = list(epoch_df.index)[0] * 60 * 60
  raw_data_length = epoch_df.shape[0] * 60 * 60
  print(raw_data_length)
  raw_df = pd.read_csv(data_dict[user_id]['raw'], skiprows=11+raw_begin_index, nrows=raw_data_length, header=None)
  raw_df.columns = ['Timestamp', 'Accelerometer X', 'Accelerometer Y', 'Accelerometer Z']

  print(raw_df.shape)

  print(epoch_df['Time_stamp'].head())
  print(raw_df.head())

  break
